In [7]:
# 导入tabulate模块，它可以将数据转换为markdown表格格式
# import tabulate
import json
import re
import os
import pandas as pd



In [2]:
## p1 = r"(?P<misson_name>.+)_(?P<gamma>\d+(\.\d+)?)_(?P<delta>.+)_(?P<threshold>.+)_z"
## 
## file = "alpacafarm_0.75_5.0_6.0_z.jsonl"
## matcher1 = re.match(p1, file)
## misson_name = matcher1.group("misson_name")
## threshold = matcher1.group("threshold")
## 
## print(misson_name, threshold)

# g0.5 d5.0
# g0.25 d5.0
# g0.75 d5.0
# g0.5 d2.0
# g0.25 d2.0
# g0.9 d5.0

In [8]:
# load data from pred


df = pd.DataFrame(columns=["model_name", "mission_name", "mode", "gamma", "delta", "threshold", "bl_type", "z_score", "sum"])

input_dir = "./pred"
p = r"(?P<model_name>.+)_(?P<mode>old|v2|gpt|new|no)_g(?P<gamma>.+)_d(?P<delta>\d+(\.\d+)?)"
p1 = r"(?P<misson_name>[a-zA-Z_]+)_(?P<gamma>\d+(\.\d+)?)_(?P<delta>.+)_z"

num = 0
# get all files from input_dir
for subfolder in os.listdir(input_dir):
    # print("subfolder is:", subfolder)
    matcher = re.match(p, subfolder)
    model_name = matcher.group("model_name")
    mode = matcher.group("mode")
    gamma = matcher.group("gamma")
    delta = matcher.group("delta")
    
    bl_type = "None"
    bl_type = (subfolder.split("_")[-1]).split(".")[0]
    
    if bl_type != "hard":
        if "old" in subfolder:
            bl_type = "soft"
        else:
            bl_type = "None"
        
        
    # print(model_name, mode, gamma, delta, bl_type)
    
    z_score_path = os.path.join(input_dir, subfolder, "z_score")
    if os.path.exists(z_score_path):
        print("subfolder is:", subfolder)
        files = os.listdir(z_score_path)
        
        for file in files:
            # print(file)
            # read jsons
            matcher1 = re.match(p1, file)
            if matcher1:
                misson_name = matcher1.group("misson_name")
                threshold = 6.0
            else:
                threshold = file.split("_")[-2]
            
            with open(os.path.join(z_score_path, file), "r") as f:
                data = json.load(f)

            # get data
            avarage_z = data["avarage_z"]
            sum = len(data["z_score_list"])
            # wm_pred_avarage = data["wm_pred_avarage"]
            
            temp = pd.DataFrame({
                "model_name": [model_name],
                "mode": [mode],
                "mission_name": [misson_name],
                "gamma": [gamma],
                "delta":[delta],
                "threshold": [threshold],
                "bl_type": [bl_type],
                "z_score": [avarage_z],
                "sum": [sum]})
            
            df = pd.concat([df, temp], ignore_index=True)
            num += 1

df.to_csv("z_score.csv")           
print(df)
print(num)
                



subfolder is: llama2-7b-chat-4k_old_g0.75_d5.0_hard
subfolder is: llama2-7b-chat-4k_v2_g0.25_d2.0
subfolder is: llama2-7b-chat-4k_old_g0.5_d2.0_hard
subfolder is: llama2-7b-chat-4k_old_g0.25_d5.0_hard
subfolder is: llama2-7b-chat-4k_gpt_g0.5_d5.0
subfolder is: llama2-7b-chat-4k_v2_g0.9_d5.0
subfolder is: llama2-7b-chat-4k_old_g0.5_d2.0
subfolder is: llama2-7b-chat-4k_gpt_g0.5_d2.0
subfolder is: llama2-7b-chat-4k_gpt_g0.9_d5.0
subfolder is: llama2-7b-chat-4k_old_g0.25_d2.0
subfolder is: llama2-7b-chat-4k_old_g0.9_d5.0_hard
subfolder is: llama2-7b-chat-4k_v2_g0.5_d5.0
subfolder is: llama2-7b-chat-4k_gpt_g0.25_d5.0
subfolder is: llama2-7b-chat-4k_v2_g0.75_d5.0
subfolder is: llama2-7b-chat-4k_v2_g0.5_d2.0
subfolder is: llama2-7b-chat-4k_old_g0.9_d5.0
subfolder is: llama2-7b-chat-4k_old_g0.5_d5.0_hard
subfolder is: llama2-7b-chat-4k_new_g0.5_d5.0
subfolder is: llama2-7b-chat-4k_no_g0.5_d5.0
subfolder is: llama2-7b-chat-4k_gpt_g0.75_d5.0
subfolder is: llama2-7b-chat-4k_old_g0.25_d2.0_hard
su

In [9]:
df_sorted = df.sort_values(by="z_score", ascending=True)
df_sorted.to_csv("z_score_sorted.csv")           
print(df_sorted)

            model_name             mission_name mode gamma delta  threshold  \
154  llama2-7b-chat-4k                      lcc   no   0.5   5.0        6.0   
137  llama2-7b-chat-4k                 hotpotqa  old   0.9   5.0        6.0   
141  llama2-7b-chat-4k  konwledge_understanding  old   0.9   5.0        6.0   
126  llama2-7b-chat-4k  konwledge_understanding   v2   0.5   2.0        6.0   
97   llama2-7b-chat-4k  konwledge_understanding  old   0.9   5.0        6.0   
..                 ...                      ...  ...   ...   ...        ...   
180  llama2-7b-chat-4k              longform_qa  old  0.25   2.0        6.0   
31   llama2-7b-chat-4k                    qmsum  old  0.25   5.0        6.0   
177  llama2-7b-chat-4k                    qmsum  old  0.25   2.0        6.0   
30   llama2-7b-chat-4k               multi_news  old  0.25   5.0        6.0   
176  llama2-7b-chat-4k               multi_news  old  0.25   2.0        6.0   

    bl_type    z_score  sum  
154    None  -0.91561